# LangChain

## Simple Chat

In [6]:
from langchain_openai import ChatOpenAI
from langchain.schema.messages import HumanMessage, AIMessage, SystemMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm_model = ChatOpenAI(
    model='deepseek-chat',
    base_url='https://api.deepseek.com',
    api_key='sk-7a3d4f4b2ca54d2dbb09e88d3572a242',
    
    streaming=True,  # 启用流式输出
    callbacks=[StreamingStdOutCallbackHandler()]  # 添加流式回调处理器
)

messages1 = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What is the weather tomorrow?")
]

response = llm_model.invoke(messages1)
# print(response.content)

I can't provide real-time weather updates, but you can check the weather for your location by using a weather service like:

- [Weather.com](https://weather.com)  
- [AccuWeather](https://www.accuweather.com)  
- [The Weather Channel](https://www.weather.gov) (for the U.S.)  
- Your smartphone's default weather app  

Let me know if you'd like help interpreting weather terms or forecasts!

## Conversation with history - Legacy（Deprecated）

In [42]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.schema.messages import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# 配置DeepSeek API
model = ChatOpenAI(
    model='deepseek-chat',
    base_url='https://api.deepseek.com',
    api_key='sk-7a3d4f4b2ca54d2dbb09e88d3572a242',
    streaming=True,  # 保留流式输出功能
    callbacks=[StreamingStdOutCallbackHandler()]  # 添加流式回调处理器
)

# 创建聊天提示模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "\n\t你是一个智能助手，与用户进行友好对话。回答尽量简洁"),
    MessagesPlaceholder(variable_name="history"),  # 存储对话历史
    ("human", "\n\t{input}")  # 当前用户输入
])

# 初始化内存
memory = ConversationBufferMemory(return_messages=True)

# 创建对话链
conversation = LLMChain(
    llm=model,
    prompt=prompt,
    memory=memory,
    verbose=False  # 显示详细的处理过程
)

# 多轮对话示例
while True:
    user_input = input("你: ")
    if user_input.lower() == "退出":
        break
    
    print(f"\n\n你: \n{user_input}")  # 确保输出立即显示
    print("\n助手: \n", end="")  # 确保输出立即显示
    # 执行对话并获取流式响应
    response = conversation.run(user_input)
    # print(f"助手: {response}")



你: 
日本的首都是什么

助手: 
东京。

## Conversation with history - Simple LCEL 
Simple LCEL with RunnableWithMessageHistory and a single built-in memory class - InMemoryChatMessageHistory

LCEL means LangChain Expression Language.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from pydantic import BaseModel, Field

model = ChatOpenAI(
    model='deepseek-chat',
    base_url='https://api.deepseek.com',
    api_key='sk-7a3d4f4b2ca54d2dbb09e88d3572a242',
    streaming=True,  # 保留流式输出功能
    callbacks=[StreamingStdOutCallbackHandler()]  # 添加流式回调处理器
)

# 创建聊天提示模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "\n\t你是一个智能助手，与用户进行友好对话。回答尽量简洁"),
    MessagesPlaceholder(variable_name="history"),  # 存储对话历史
    ("human", "\n\t{input}")  # 当前用户输入
])

memory = InMemoryChatMessageHistory()

chain = prompt | model
chain = RunnableWithMessageHistory(
    chain,
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="history",
)

# 多轮对话示例
while True:
    user_input = input("你: ")
    if user_input.lower() == "退出":
        break
    
    print(f"\n\n你: \n{user_input}")  # 确保输出立即显示
    print("\n助手: \n", end="")  # 确保输出立即显示
    chain.invoke({"input": user_input}, config={"configurable": {"session_id": "123"}})



你: 
日本在哪里

助手: 
日本位于东亚，是一个岛国，在太平洋西北部，与中国、韩国隔海相望。

你: 
有哪些美食

助手: 
日本美食很多，主要有：  
- **寿司** 🍣  
- **拉面** 🍜  
- **天妇罗** 🍤  
- **刺身**（生鱼片）  
- **章鱼烧** 🐙  
- **和牛** 🥩  
- **抹茶甜点** 🍵  

都很有特色，值得一试！ 😋

## Conversation with history - LCEL with sessions
LCEL with RunnableWithMessageHistory and self-implemented memory class - InMemoryHistory

In [50]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.messages import BaseMessage, AIMessage
from langchain_core.runnables.history import RunnableWithMessageHistory
from pydantic import BaseModel, Field

model = ChatOpenAI(
    model='deepseek-chat',
    base_url='https://api.deepseek.com',
    api_key='sk-7a3d4f4b2ca54d2dbb09e88d3572a242',
    streaming=True,  # 保留流式输出功能
    callbacks=[StreamingStdOutCallbackHandler()]  # 添加流式回调处理器
)

# 创建聊天提示模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "\n\t你是一个智能助手，与用户进行友好对话。回答尽量简洁"),
    MessagesPlaceholder(variable_name="history"),  # 存储对话历史
    ("human", "\n\t{input}")  # 当前用户输入
])

class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: list[BaseMessage]) -> None:
        """Add a list of messages to the store"""
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []

store = {}

def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

chain = prompt | model
chain = RunnableWithMessageHistory(
    chain,
    get_by_session_id,
    input_messages_key="input",
    history_messages_key="history",
)

# 多轮对话示例
while True:
    user_input = input("你: ")
    if user_input.lower() == "退出":
        break
    
    print(f"\n\n你: \n{user_input}")  # 确保输出立即显示
    print("\n助手: \n", end="")  # 确保输出立即显示
    chain.invoke({"input": user_input}, config={"configurable": {"session_id": "123"}})



你: 
委内瑞拉的首都是什么

助手: 
加拉加斯。

你: 
有哪些美食

助手: 
委内瑞拉特色美食包括：  

1. **Arepa**（玉米饼）—— 用玉米面做的饼，可夹各种馅料。  
2. **Pabellón Criollo**（国菜）—— 碎牛肉、黑豆、米饭和炸芭蕉的组合。  
3. **Cachapa**（甜玉米饼）—— 类似薄煎饼，常配奶酪。  
4. **Tequeños**（芝士条）—— 酥皮包裹芝士，油炸小吃。  
5. **Hallaca**（圣诞粽子）—— 芭蕉叶包裹的玉米面团，内含肉馅。  

简单又美味！ 😊

## Structured Output (JSON) With Prompt Engineering

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.callbacks import StreamingStdOutCallbackHandler

llm_model = ChatOpenAI(
    model="deepseek-chat",
    base_url='https://api.deepseek.com',
    api_key='sk-7a3d4f4b2ca54d2dbb09e88d3572a242',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

messages = [
    SystemMessage(content="You are a tool assistant, your response is a pure json output."),
    HumanMessage(content="what is the weather and temperature in beijing today?")
]

llm_model(messages)

```json
{
  "response": {
    "location": "Beijing",
    "date": "2023-11-02",
    "weather": "Sunny",
    "temperature": {
      "current": "15°C",
      "high": "18°C",
      "low": "8°C"
    },
    "humidity": "30%",
    "wind": {
      "speed": "10 km/h",
      "direction": "North"
    },
    "air_quality": {
      "index": "85",
      "level": "Moderate"
    }
  }
}
```

AIMessage(content='```json\n{\n  "response": {\n    "location": "Beijing",\n    "date": "2023-11-02",\n    "weather": "Sunny",\n    "temperature": {\n      "current": "15°C",\n      "high": "18°C",\n      "low": "8°C"\n    },\n    "humidity": "30%",\n    "wind": {\n      "speed": "10 km/h",\n      "direction": "North"\n    },\n    "air_quality": {\n      "index": "85",\n      "level": "Moderate"\n    }\n  }\n}\n```', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--f4ef3b40-57cb-4af9-a2a4-b63363895291-0')

## Structured Output (JSON) With JsonOutputParser

In [ ]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain.output_parsers import OutputFixingParser
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.callbacks import StreamingStdOutCallbackHandler

model = ChatOpenAI(
    model='deepseek-chat',
    base_url='https://api.deepseek.com',
    api_key='sk-7a3d4f4b2ca54d2dbb09e88d3572a242',
    streaming=True,  # 保留流式输出功能
    callbacks=[StreamingStdOutCallbackHandler()]  # 添加流式回调处理器
)

class BuildingInfo(BaseModel):
    name: str = Field(description="Building name")
    address: str = Field(description="Building address")
    city: str = Field(description="City where the building is located")
    country: str = Field(description="Country where the building is located")
    completion_year: int = Field(description="Year of completion of the building")

parser = JsonOutputParser(pydantic_object=BuildingInfo)
# optional, for fixing the output 
parser = OutputFixingParser.from_llm(parser=parser, llm=model, max_retries=3) 

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# parser.get_format_instructions() is an encapsulated prompt engineering method for guiding an LLM's to return structured output
print("format_instructions: ", parser.get_format_instructions())
print("*" * 50)
print("prompt: ", prompt)
print("*" * 50)

# RunnableSerializable Chaining Format:
chain = prompt | model | parser
x = {"query":"Tell me about the Empire State Building."}
response = chain.invoke(x)

print("\nDict Object: ", response)
print("name: ", response["name"])
print("completion_year: ", response["completion_year"])

# print("*" * 50)

# # Normal Chaining Format:
# x = {"query":"Tell me about the Empire State Building."}
# x = prompt.invoke(x)
# x = model.invoke(x)
# response = parser.invoke(x) # or parser.parse(x.content)

# print("\nDict Object: ", response)
# print("name: ", response["name"])
# print("completion_year: ", response["completion_year"])

format_instructions:  The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "Building name", "title": "Name", "type": "string"}, "address": {"description": "Building address", "title": "Address", "type": "string"}, "city": {"description": "City where the building is located", "title": "City", "type": "string"}, "country": {"description": "Country where the building is located", "title": "Country", "type": "string"}, "completion_year": {"description": "Year of completion of the building", "title": "Completion Year", "type": "integer"}}, "required": ["name", 

## Tool Calling with bind_tools - Get Weather

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool

model = ChatOpenAI(
    model='deepseek-chat',
    base_url='https://api.deepseek.com',
    api_key='sk-7a3d4f4b2ca54d2dbb09e88d3572a242',
    streaming=True,  # 保留流式输出功能
    callbacks=[StreamingStdOutCallbackHandler()]  
)

@tool
def get_wheather(location: str):
    """
    Function to get the weather for a given location.
    """
    weather = {
        "New York": "sunny",
        "Shanghai": "cloudy",
        "Seoul": "rainy",
    }
    print(f"Getting weather for {location}")
    return weather.get(location, "unknown")

tools = { "get_wheather": get_wheather }
model_with_tools = model.bind_tools([get_wheather])

messages = [HumanMessage("What's the weather in New York?"),]
response = model_with_tools.invoke(messages)
messages.append(response)
print(messages)

for tool_call in response.tool_calls:
    selected_tool = tools[tool_call["name"]]
    tool_msg = selected_tool.invoke(tool_call) #get_wheather(location="New York")
    messages.append(tool_msg)

print(messages)
response = model_with_tools.invoke(messages)
print(response)


[HumanMessage(content="What's the weather in New York?", additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_0_d283731e-064a-43d5-be73-da7eed4573dd', 'function': {'arguments': '{"location":"New York"}', 'name': 'get_wheather'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0425fp8'}, id='run--a9d44500-0bc1-42de-8648-1568f8a25b8c-0', tool_calls=[{'name': 'get_wheather', 'args': {'location': 'New York'}, 'id': 'call_0_d283731e-064a-43d5-be73-da7eed4573dd', 'type': 'tool_call'}], usage_metadata={'input_tokens': 115, 'output_tokens': 21, 'total_tokens': 136, 'input_token_details': {'cache_read': 64}, 'output_token_details': {}})]
Getting weather for New York
[HumanMessage(content="What's the weather in New York?", additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'inde

# LangGraph

## Agent - Booking Hotel and Flights

In [1]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from typing import Annotated
import json
import requests
from langgraph.checkpoint.memory import InMemorySaver
import datetime
import pytz

MODEL_URL = 'https://api.deepseek.com'
MODEL_API_KEY = 'sk-7a3d4f4b2ca54d2dbb09e88d3572a242'

BASE_URL = "https://serpapi.com/search"
SERPER_API_KEY='39e13214c84bb203818c0f76ce4552fdb310538fd92a2026c93561dc54ff9bff'

def search_hotels(
    query: Annotated[str, "The city name or hotel name to search"], 
    check_in_date: Annotated[str, "Hotel Check-in date in YYYY-MM-DD format"], 
    check_out_date: Annotated[str, "Hotel Check-out date in YYYY-MM-DD format"],
) -> Annotated[str, "Returns a JSON string containing the search results."]:
    """
    Function to search hotels based on the given query, check-in date, and check-out date.
    """
    params = {
        "api_key": SERPER_API_KEY,
        "engine": "google_hotels",
        "q": query,
        "check_in_date": check_in_date,
        "check_out_date": check_out_date,
        "currency": "USD",
        "hl": "en",
        "gl": "uk",
    }
    print("Calling search_hotels with params:", {"query": query, "check_in_date": check_in_date, "check_out_date": check_out_date})
    response = requests.get(BASE_URL, params=params)
    result = response.json()
    print(response)
    return json.dumps(result)

def search_flights(
    departure_id: Annotated[str, "Departure id in IATA format"], 
    arrival_id: Annotated[str, "Arrival id in IATA format"],
    outbound_date: Annotated[str, "Flight outbound date in YYYY-MM-DD format"],
    return_date: Annotated[str, "Flight return date in YYYY-MM-DD format"],
) -> Annotated[str, "Returns a JSON string containing the search results."]:
    """
    Function to search flights
    """
    params = {
        'api_key': SERPER_API_KEY,
        "engine": "google_flights",
        "departure_id": departure_id,
        "arrival_id": arrival_id,
        "outbound_date": outbound_date,
        "return_date": return_date,
        "currency": "USD",
        "hl": "en",
        "gl": "uk",
    }
    print("Calling search_flights with params:", {"departure_id": departure_id, "arrival_id": arrival_id, "outbound_date": outbound_date, "return_date": return_date})
    response = requests.get(BASE_URL, params=params)
    result = response.json()
    print(response)
    return json.dumps(result)

def get_curent_datetime(
    zone: Annotated[str, "Timezone in pytz format to get the current datetime"],
) -> Annotated[str, "Returns the current datetime in '%Y-%m-%d %H:%M:%S %Z%z' format."]:
    """
    Function to get the current datetime in the given timezone.
    """
    print("Calling get_curent_datetime with timezone:", zone)
    local_timezone = pytz.timezone(zone)
    now = datetime.datetime.now(local_timezone).strftime("%Y-%m-%d %H:%M:%S %Z%z")
    print("Current datetime in", zone, ":", now)
    return now

prompt = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If you want to know the current time and date in a specific timezone, for example the date of tomorrow in the place of departure, use the get_curent_datetime API to call.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
"""

model = ChatOpenAI(
    model='deepseek-chat',
    base_url=MODEL_URL,
    api_key=MODEL_API_KEY,
    streaming=True,  # 保留流式输出功能
    callbacks=[StreamingStdOutCallbackHandler()]  # 添加流式回调处理器
)

checkpointer = InMemorySaver()

agent = create_react_agent(
    model=model,
    tools=[search_hotels, search_flights, get_curent_datetime],
    checkpointer=checkpointer,
    prompt=prompt,
)

while True:
    user_input = input("You: ")
    if user_input.lower() == "bye":
        break
    
    print(f"\n\nYou: \n{user_input}")
    print("\nAssistant: \n", end="") 
    
    agent.invoke(
        {"messages": user_input},
        config={"configurable": {"thread_id": "1"}}
    )

# Example prompts:
# 1. I want to fly from Guangzhou to Seoul the morning after tomorrow.
# 1. I want to fly from New York to Seoul tomorrow morning.
# 2. I am going to stay there for 3 days, book a hotel for me, the cheapest one.
# 3. Just show me the best flight and hotel. you decide it for me.



You: 
I want to fly from Guangzhou to Seoul the morning after tomorrow.

Assistant: 
Calling get_curent_datetime with timezone: Asia/Shanghai
Current datetime in Asia/Shanghai : 2025-05-23 20:16:07 CST+0800
Calling search_flights with params: {'departure_id': 'CAN', 'arrival_id': 'ICN', 'outbound_date': '2025-05-25', 'return_date': ''}
<Response [400]>
Calling search_flights with params: {'departure_id': 'CAN', 'arrival_id': 'ICN', 'outbound_date': '2025-05-25', 'return_date': '2025-05-26'}
<Response [200]>
Here are the available flight options from Guangzhou (CAN) to Seoul (ICN) for the morning after tomorrow (May 25, 2025):

### Outbound Flights (May 25, 2025)

| Departure Airport Name       | Airline          | Flight Number | Departure Time | Arrival Airport Name       | Arrival Time | Duration | Airplane      | Travel Class | Price (USD) | Legroom | Extensions                                  | Carbon Emissions (kg) |
|------------------------------|------------------|----------